# SRCNN

在水文水资源领域，降尺度等问题和计算机视觉中的超分辨率问题有相似性，因此相关的方法也可以应用到专业领域，因此本文目的是了解SRCNN的基本概念，以更好地理解相关专业的论文。

主要参考资料有：

- [从SRCNN到EDSR，总结深度学习端到端超分辨率方法发展历程](https://zhuanlan.zhihu.com/p/31664818)
- [经典论文复现：基于深度学习的图像超分辨率重建](https://zhuanlan.zhihu.com/p/45627373)
- [机器学习——字典学习/稀疏编码学习笔记](https://zhuanlan.zhihu.com/p/26015351)
- [Dictionary Learning(字典学习、稀疏表示以及其他)](https://www.cnblogs.com/hdu-zsk/p/5954658.html)
- [字典学习（Dictionary Learning, KSVD）详解](https://www.cnblogs.com/endlesscoding/p/10090866.html)
- [【《数学之美》笔记（一）】奇异值分解(SVD)的原理、演算和应用](https://zhuanlan.zhihu.com/p/64709797)
- [深入浅出字典学习（Dictionary Learning）](https://www.jianshu.com/p/f6e5d1cd21b9)
- [[图像重建算法] 基于稀疏编码的图像超分辨](https://zhuanlan.zhihu.com/p/82749548)
- [基于稀疏表示的图像超分辨](https://blog.csdn.net/qq_31339017/article/details/76177366)
- [基于K近邻稀疏编码均值约束的人脸超分辨率算法](http://www.jsjkx.com/CN/article/openArticlePDF.jsp?id=6036)
- [yiyang7/Super_Resolution_with_CNNs_and_GANs](https://github.com/yiyang7/Super_Resolution_with_CNNs_and_GANs)

超分辨率技术（Super-Resolution, SR）是指从观测到的低分辨率图像重建出相应的高分辨率图像，在监控设备、卫星图像和医学影像等领域都有重要的应用价值。

本文针对端到端的基于深度学习的单张图像超分辨率方法(Single Image Super-Resolution, SISR)，总结一下从SRCNN到EDSR的发展历程。(排列顺序大致按论文中给出的4倍上采样结果的峰值信噪比(Peak Signal to Noise Ratio, PSNR)从低到高排列)

## SRCNN

SRCNN是深度学习用在超分辨率重建上的开山之作。

单图像超分辨率重构（SR）可以**从一张较小的图像生成一张高分辨率的图像**。显然，这种恢复的**结果是不唯一的**，相当于方程数少于变量数。

可以这样直观地理解：远远看到一个模糊的身影，看不清脸，既可以认为对面走来的是个男生，也可以认为这是个女生。那么，当我想象对面人的长相时，会如何脑补呢？

这就依赖于我们的**先验知识**。假如我认为，一个穿着裙子的人肯定是个女生，而对面那个人穿着裙子，所以我认为那是个女生，脑补了一张女神脸。然而，如果我知道穿裙子的人不一定是女生，还可能是女装大佬。迎面走来那个人瘦瘦高高，所以我认为十有八九是个男孩子，就会脑补一个……也就是说，**不同的先验知识，会指向不同的结果**。我们的任务，就是**学习这些先验知识**。目前效果最好的办法**都是基于样本的（example-based）**。

论文提出一种有趣的视角：**CNN 所构造的模型和稀疏编码方法（sparse coding based）是等价的**。

先简单补充下稀疏编码方法的基本概念，然后再看这句话什么意思。

### 稀疏编码方法基本概念

字典学习（Dictionary Learning）和稀疏表示（Sparse Representation）在学术界的正式称谓应该是稀疏字典学习（Sparse Dictionary Learning）。该算法理论包含两个阶段：**字典构建阶段（Dictionary Generate）** 和 **利用字典（稀疏的）表示样本阶段（Sparse coding with a precomputed dictionary）**。这两个阶段（如下图）的每个阶段都有许多不同算法可供选择，每种算法的诞生时间都不一样，以至于稀疏字典学习的理论提出者已变得不可考。

![](808068-20161012222200109-1848074195.png)

为什么需要字典学习？

无论人类的知识有多么浩繁，也无论人类的科技有多么发达，一本新华字典或牛津字典足以表达人类从古至今乃至未来的所有知识，那些知识只不过是字典中字的排列组合罢了！直到这里，我相信相当一部分读者或许在心中已经明白了字典学习的第一个好处

（1）它实质上是**对于庞大数据集的一种降维表示**，或者说是信息的压缩
（2）正如同字是句子最质朴的特征一样，字典学习总是尝试**学习蕴藏在样本背后最质朴的特征**（假如样本最质朴的特征就是样本最好的特征）。

为什么需要稀疏表示？

再举一个例子。相信大部分人都有这样一种感觉，当我们在解涉及到新的知识点的数学题时总有一种累心（累脑）的感觉。但是当我们通过艰苦卓绝的训练将新的知识点牢牢掌握时，再解决与这个知识点相关的问题时就不觉得很累了。这是为什么呢？意大利罗马大学的Fabio Babiloni教授曾经做过一项实验，他们让新飞行员驾驶一架飞机并采集了他们驾驶状态下的脑电，同时又让老飞行员驾驶飞机并也采集了他们驾驶状态下的脑电。

![](v2-24aa7812f5b64d6a0e6fc4125713dab8_720w.png)

结论是熟练者的大脑（右图）可以**调动尽可能少的脑区消耗尽可能少的能量进行同样有效的计算**（所以熟悉知识点的你，大脑不会再容易觉得累了），并且由于调动的脑区很少，大脑计算速度也会变快，这就是我们称熟练者为熟练者的原理所在。站在我们所要理解的稀疏字典学习的角度上来讲就是大脑学会了知识的稀疏表示。

稀疏表示的本质：**用尽可能少的资源表示尽可能多的知识，这种表示还能带来一个附加的好处，即计算速度快**。

适应性（泛化能力）和稀疏性之间找平衡，最优取决于代价函数。

在计算机中，“字典”和“稀疏”却是一堆孪生兄弟。在学习样本字典之初的时候，稀疏条件就已经被加入了。我们希望字典里的字可以尽能的少，但是却可以尽可能的表示最多的句子。这样的字典最容易满足稀疏条件。也就是说，这个“字典”是这个“稀疏”私人订制的。

再扯两点：

1）特征选择(Feature Selection)

大家对稀疏规则化趋之若鹜的一个关键原因在于它能实现**特征的自动选择**。一般来说，$x_i$ 的大部分元素（也就是特征）都是和最终的输出 $y_i$ 没有关系或者不提供任何信息的，在最小化目标函数的时候考虑$x_i$这些额外的特征，虽然可以获得更小的训练误差，但在预测新的样本时，这些没用的信息反而会被考虑，从而干扰了对正确$y_i$的预测。稀疏规则化算子的引入就是为了完成特征自动选择的光荣使命，它会**学习地去掉这些没有信息的特征**，也就是把这些特征对应的权重置为0。

2）可解释性(Interpretability)

另一个青睐于稀疏的理由是，模型更容易解释。例如患某种病的概率是y，然后我们收集到的数据x是1000维的，也就是我们需要寻找这1000种因素到底是怎么影响患上这种病的概率的。假设我们这个是个回归模型：$y=w_1*x_1+w_2*x_2+\cdots+w_{1000}*x_{1000}+b$（当然了，为了让y限定在[0,1]的范围，一般还得加个Logistic函数）。通过学习，如果最后学习到的$w_*$就只有很少的非零元素，例如只有5个非零的$w_i$，那么我们就有理由相信，这些对应的特征在患病分析上面提供的信息是巨大的，决策性的。也就是说，患不患这种病只和这5个因素有关，那医生就好分析多了。但如果1000个$w_i$都非0，医生面对这1000种因素，累觉不爱。

在计算机视觉领域，稀疏编码有很好的发展。

1995 年前后，Bruno Olshausen和 David Field 两位学者任职 Cornell University，他们试图同时用生理学和计算机的手段，双管齐下，研究视觉问题。

他们收集了很多黑白风景照片，从这些照片中，提取出400个小碎片，每个照片碎片的尺寸均为 16x16 像素，不妨把这400个碎片标记为 S[i], i = 0,.. 399。

接下来，再从这些黑白风景照片中，随机提取另一个碎片，尺寸也是 16x16 像素，不妨把这个碎片标记为 T。

他们提出的问题是，如何从这400个碎片中，选取一组碎片，S[k], 通过叠加的办法，合成出一个新的碎片，而这个新的碎片，应当与随机选择的目标碎片 T，尽可能相似，同时，S[k] 的数量尽可能少。用数学的语言来描述，就是：

$\sum_k (a[k] * S[k]) --> T$, 其中 a[k] 是在叠加碎片 S[k] 时的权重系数。

为解决这个问题，Bruno Olshausen和 David Field 发明了一个算法，稀疏编码（Sparse Coding）。

稀疏编码是一个重复迭代的过程，每次迭代分两步：

1. 选择一组 S[k]，然后调整 a[k]，使得$\sum_k (a[k] * S[k])$ 最接近 T。
2. 固定住 a[k]，在 400 个碎片中，选择其它更合适的碎片S'[k]，替代原先的 S[k]，使得$\sum_k (a[k] * S'[k])$ 最接近 T。

经过几次迭代后，最佳的 S[k] 组合，被遴选出来了。令人惊奇的是，被选中的 S[k]，基本上都是照片上不同物体的边缘线，这些线段形状相似，区别在于方向。

Bruno Olshausen和 David Field 的算法结果，与 David Hubel 和Torsten Wiesel 的生理发现，不谋而合！也就是说，**复杂图形，往往由一些基本结构组成**。

以上是一些基本概念，接下来再进一步了解下字典学习和稀疏表示的基本数学表达。

### 稀疏编码数学基本表达

字典学习的思想应该源来实际生活中的字典的概念。字典是前辈们学习总结的精华，当我们需要学习新的知识的时候，不必与先辈们一样去学习先辈们所有学习过的知识，我们可以参考先辈们给我们总结的字典，通过查阅这些字典，我们可以大致学会到这些知识。

为了将上述过程用准确的数学语言描述出来，我们需要将 **“总结字典”、“查阅字典”** 做出一个更为准确的描述。就从我们的常识出发：

我们通常会要求的我们的**字典尽可能全面**，也就是说总结出的字典不能漏下关键的知识点。查字典的时候，我们想要我们查字典的过程尽可能**简洁，迅速，准确**。查到的结果，要**尽可能地还原出原来知识**。当然，如果要完全还原出来，那么这个字典和查字典的方法会变得非常复杂，所以我们只需要尽可能地还原出原知识点即可。

将上面的所提到的关键点用几个数学符号表示一下：

- “以前的知识”，更专业一点，我们称之为原始样本，用矩阵Y表示；
- “字典”，我们称之为字典矩阵，用D表示，“字典”中的词条，我们称之为原子（atom），用列向量$d_k$表示；
- “查字典的方法”，我们称为稀疏矩阵，用X；
- “查字典的过程”，我们可以用矩阵的乘法来表示，即DX。

用数学语言描述，字典学习的主要思想是，利用包含K个原子$d_k$的字典矩阵$D\in R^{m*K}$，稀疏线性表示原始样本 $Y \in R^{m*n}$（其中m表示样本数，n表示样本的属性），即有Y=DX（这只是理想的情况），其中$X\in R^{K*n}$为稀疏矩阵。则问题就是：
$$min_{D,X}||Y-DX||^2_F, s.t. \forall i, ||x_i||_0 \le T_0$$

上式中X为稀疏编码的矩阵，$x_i$ (i=1,2,⋯,K)为该矩阵中的行向量，代表字典矩阵的系数。
$||x_i||_0$表示零阶范数，即向量中不为0的数的个数。

这个目标函数表示，我们要最小化查完的字典与原始样本的误差，即要尽可能还原出原始样本；它的限的制条件$||x_i||\le T_0$，表示查字典的方式要尽可能简单，即X要尽可能稀疏。

这是一个带有约束的优化问题，可以利用拉格朗日乘子法将其转化为无约束优化问题，具体求解过程就先省略了，后面实际遇到再分析，总之，利用奇异值分解等算法，可以求解。比如字典学习的K-SVD算法。

In [11]:
import numpy as np
import pandas as pd
from scipy.io import loadmat
 
train_data_mat = loadmat("160px-Real_Madrid_CF.mat")
 
train_data = train_data_mat["M"][:,:,0]
train_label = np.array(["RM"])
 
print(train_data.shape, train_label.shape)

(224, 160) (1,)


初始化字典

In [12]:
u, s, v = np.linalg.svd(train_data)
n_comp = 50
dict_data = u[:, :n_comp]

字典更新

In [13]:
def dict_update(y, d, x, n_components):
    """
    使用KSVD更新字典的过程
    """
    for i in range(n_components):
        index = np.nonzero(x[i, :])[0]
        if len(index) == 0:
            continue
        # 更新第i列
        d[:, i] = 0
        # 计算误差矩阵
        r = (y - np.dot(d, x))[:, index]
        # 利用svd的方法，来求解更新字典和稀疏系数矩阵
        u, s, v = np.linalg.svd(r, full_matrices=False)
        # 使用左奇异矩阵的第0列更新字典
        d[:, i] = u[:, 0]
        # 使用第0个奇异值和右奇异矩阵的第0行的乘积更新稀疏系数矩阵
        for j,k in enumerate(index):
            x[i, k] = s[0] * v[0, j]
    return d, x

迭代更新求解#
可以指定迭代更新的次数，或者指定收敛的误差。

In [15]:
from sklearn import linear_model
 
max_iter = 10
dictionary = dict_data
 
y = train_data
tolerance = 1e-6
 
for i in range(max_iter):
    # 稀疏编码
    x = linear_model.orthogonal_mp(dictionary, y)
    e = np.linalg.norm(y - np.dot(dictionary, x))
    if e < tolerance:
        break
    dict_update(y, dictionary, x, n_comp)
 
sparsecode = linear_model.orthogonal_mp(dictionary, y)
 
train_restruct = dictionary.dot(sparsecode)
train_restruct

array([[-6.79939512e-02, -1.31142685e-01, -7.54645832e-01, ...,
        -8.04738139e-01, -1.65474092e-01, -6.56052946e-02],
       [-1.52113231e-01, -2.42888849e-01, -1.79463900e+00, ...,
        -1.94590425e+00, -2.53375986e-01, -1.46445899e-01],
       [-1.08865244e-01, -5.40177455e-01, -1.30589742e+00, ...,
        -1.41684226e+00, -5.54649707e-01, -1.05059218e-01],
       ...,
       [-6.69285476e-03,  2.90669357e+00, -6.27181325e+00, ...,
        -8.35837232e+00,  2.12445030e+00,  2.09344994e-02],
       [-3.27711717e-01, -4.39781630e-01, -3.82688678e+00, ...,
        -4.13299666e+00, -5.65045785e-01, -3.01511857e-01],
       [-2.98440713e-02, -3.71274064e-01, -1.83001881e-01, ...,
        -1.41698578e-01, -8.17784463e-01, -2.83806798e-02]])

简单概述下，对于图片来说就是通过稀疏表达的方式对原有图像数据进行压缩，同时保证图像尽量不失真。

![](3728828-7095a84e66b31f85.webp)

y向量代表原有的图像（640000维），A是字典矩阵（K\*640000），X是稀疏表示向量（K维），因为K远远小于N，我们认为，稀疏表示后的数据获得了大幅的压缩。**求A的过程通常称为字典学习**。**已知A，求X的过程称为稀疏表示**。通常这两者可以等同。在实际训练的过程中，为了减少计算量，通常将图像切割为小的patch（8\*8或16\*16），然后针对这些patch进行训练获得patch字典。

### 稀疏编码与SR

上面简单了解了稀疏编码后，需要进一步了解这和SR问题是怎么联系到一起的。方法挺多，可以搜索相关文献，这里首先结合博客对Image Super-Resolution via Sparse Representation这篇文章的基本逻辑做些记录。然后再看看K近邻算法和SR问题的关系。

首先第一种基本逻辑是这样的，联合训练两个用于低分辨率和高分辨率图像块的字典，令**低分辨率和高分辨率图像块对之间的稀疏表示相等**（全耦合模式），那么低分辨图像的利用低分辨字典稀疏表示后直接替换成相应的高分辨字典基元即可合成一张高分辨图像。

设高分辨图像块展开成一位数列 $x \in \mathfrak{R}^n$ 可以由一组K个基元（atom）的过完备字典 $D\in \mathfrak{R}^{n*K}$ 线性稀疏表示。其中 $a_0 \in \mathfrak{R}^K$ 是表征向量
$$x=D\alpha_0$$
设投影向量 $L \in \mathfrak{R}^{k*n}$ ，将高分辨图像块 x 变为低分辨图像块 y
$$y=Lx=LD\alpha_0$$
提取相对最稀疏的向量 $\alpha_0$作为**表征向量**。

文章并没有直接学习高分辨图像块的稀疏表征，而是利用两组耦合字典 $D_h$ 和 $D_l$ 分别作为高分辨和低分辨字典，对应的低分辨图像块的表征向量与高分辨图像块的表征向量相同（很重要的前提假设）！

b)标记和记号： X 和 Y 分别表示高分辨/低分辨图像，小写的 x 和 y 表示高分辨/低分辨图像块， $D_h$ 和 $D_l$ 分别表示高分辨/低分辨字典， S 是下采样操作。

稀疏编码的超分辨问题是一个约束优化问题，前面提到了这个问题包含着稀疏项表示：
$$x\approx D_h\alpha$$
其中，$\alpha \in \mathfrak{R}^K$, $||\alpha||_0 << K$

首先学习稀疏表征，然后利用低分辨的表征重建高分辨图像，最后通过保真约束继续优化一下，最后这一步先不用管。也就是说先学习稀疏表征，这个分别对低分辨率和高分辨率的图像分别学习，然后重建（即低到高）。

稀疏表示的局部模型是
$$min_{\alpha} ||FD_l\alpha-Fy||^2_2+\lambda||\alpha||_1$$
$$min_{\alpha} ||PD_h\alpha-\omega||^2_2+\lambda||\alpha||_1$$
其中，F是特征提取操作，$\lambda$是拉格朗日乘子，P提取当前目标图像块和原始重建高分辨率图像重叠部分，$\omega$表示重叠部分中原始重建高分辨率图像的像素值。则两式可合在一起
$$min_{\alpha} ||\tilde D\alpha-\tilde y||^2_2+\lambda||\alpha||_1$$
这些可以得到最优的稀疏表示稀疏$\alpha ^*$，这样高分辨率图像块就可以被重建为:
$$x=D_h\alpha^*$$

接下来是K近邻稀疏编码。其属于一种基于局部非参数模型的方法，基本思路是首先将人脸图像划分成大小相等的图像块，然后针对每个小的图像块进行超分辨率重建，最后将重建后的高分辨率图像块融合在一起，得到完整的高分辨率人脸图像。

首先将人脸图像划分成大小相等的图像块，然后针对每个小的图像块进行超分辨率重建，最后将重建后的高分辨率图像块融合在一起 ，得到完整的高分辨率人脸图像。

由于这类方法能够充分利用局部人脸图像的统计特性，如眼睛、嘴、位置等，较之输入图像，合成的高分辨率人脸图像能够明显地改善输人图像的清晰度。不过 ，该方法也存在着易受噪声干扰且计算量大的弊端。通常最近邻方法被用于**减少基于局部非参数模型的人脸超分辨率方法的计算复杂度**。

暂时理解不了，还需要补充相关资料。

### SRCNN 结构

以上简单了解了稀疏编码，现在回头看看“CNN 所构造的模型和稀疏编码方法（sparse coding based）是等价的”的含义。

稀疏编码方法的流程如下：

1. 从原始图片中切割出一个个小块，并进行预处理（归一化）。这种切割是密集的，也就是**块与块之间有重叠**；
2. 使用**低维词典（low-resolution dictionary）编码**，得到一个稀疏参数；
3. 使用**高维词典（high-resolution dictionary）结合稀疏参数进行重建（换了个密码本）**；
4. 将多个小块拼接起来，重合部分使用加权和拼接。

SRCNN的网络结构简单，仅仅用了三个卷积层，网络结构如下图所示。

![](v2-48339af4c2ac2ad7f858eecf513dfacd_720w.jpg)

本文中，作者将三层卷积的结构解释成三个步骤：图像块的提取和特征表示，特征非线性映射和最终的重建。这是卷积神经网络对应于稀疏编码的结构。对于一个低分辨率图像 Y，第一个卷积层提取 feature maps。第二个卷积层将 feature maps 进行非线性变换，变换为高分辨率图像的表示。最后一层恢复出高分辨率图像。

总之，SRCNN首先使用双三次(bicubic)插值将低分辨率图像放大成目标尺寸，接着通过三层卷积网络拟合非线性映射，最后输出高分辨率图像结果。

三个卷积层使用的卷积核的大小分为为9x9,，1x1和5x5，前两个的输出特征个数分别为64和32。用Timofte数据集（包含91幅图像）和ImageNet大数据集进行训练。使用均方误差(Mean Squared Error, MSE)作为损失函数，有利于获得较高的PSNR。

代码：[Image Super-Resolution Using Deep Convolutional Networks](http://mmlab.ie.cuhk.edu.hk/projects/SRCNN.html)

## FSRCNN

FSRCNN与SRCNN都是香港中文大学Dong Chao， Xiaoou Tang等人的工作。FSRCNN是对之前SRCNN的改进，主要在三个方面：一是在最后使用了一个反卷积层放大尺寸，因此可以直接将原始的低分辨率图像输入到网络中，而不是像之前SRCNN那样需要先通过bicubic方法放大尺寸。二是改变特征维数，使用更小的卷积核和使用更多的映射层。三是可以共享其中的映射层，如果需要训练不同上采样倍率的模型，只需要fine-tuning最后的反卷积层。

由于FSRCNN不需要在网络外部进行放大图片尺寸的操作，同时通过添加收缩层和扩张层，将一个大层用一些小层来代替，因此FSRCNN与SRCNN相比有较大的速度提升。FSRCNN在训练时也可以只fine-tuning最后的反卷积层，因此训练速度也更快。FSRCNN与SCRNN的结构对比如下图所示。

![](v2-ee451a707399625eee3e194f7a9d6ca0_720w.jpg)

FSRCNN可以分为五个部分。特征提取：SRCNN中针对的是插值后的低分辨率图像，选取的核大小为9×9，这里直接是对原始的低分辨率图像进行操作，因此可以选小一点，设置为5×5。收缩：通过应用1×1的卷积核进行降维，减少网络的参数，降低计算复杂度。非线性映射：感受野大，能够表现的更好。SRCNN中，采用的是5×5的卷积核，但是5×5的卷积核计算量会比较大。用两个串联的3×3的卷积核可以替代一个5×5的卷积核，同时两个串联的小卷积核需要的参数3×3×2=18比一个大卷积核5×5=25的参数要小。FSRCNN网络中通过m个核大小为3×3的卷积层进行串联。扩张：作者发现低维度的特征带来的重建效果不是太好，因此应用1×1的卷积核进行扩维，相当于收缩的逆过程。反卷积层：可以堪称是卷积层的逆操作，如果步长为n，那么尺寸放大n倍，实现了上采样的操作。

FSRCNN中激活函数采用PReLU，损失函数仍然是均方误差。对CNN来说，Set91并不足够去训练大的网络结构，FSRCNN提出general-100 + Set91进行充当训练集。并且进行数据增强，1）缩小尺寸为原来的0.9, 0.8, 0.7和0.6。2）旋转 90°，180°和270°，因此获得了数据量的提升。

代码：[Accelerating the Super-Resolution Convolutional Neural Network](http://mmlab.ie.cuhk.edu.hk/projects/FSRCNN.html)

## ESPCN

作者在本文中介绍到，像SRCNN那样的方法，由于需要将低分辨率图像通过上采样插值得到与高分辨率图像相同大小的尺寸，再输入到网络中，这意味着要在较高的分辨率上进行卷积操作，从而增加了计算复杂度。本文提出了一种直接在低分辨率图像尺寸上提取特征，计算得到高分辨率图像的高效方法。ESPCN网络结构如下图所示。

![](v2-077f35d5416b8f2bf38eaf888201a0b7_720w.jpg)

ESPCN的核心概念是亚像素卷积层(sub-pixel convolutional layer)。网络的输入是原始低分辨率图像，通过三个卷积层以后，得到通道数为 $r^2$ 的与输入图像大小一样的特征图像。再将特征图像每个像素的 $r^2$ 个通道重新排列成一个 $r*r$ 的区域，对应高分辨率图像中一个 $r*r$ 大小的子块，从而大小为 $H*W*r^2$ 的特征图像被重新排列成 $rH*rW*1$ 的高分辨率图像。我理解的亚像素卷积层包含两个过程，一个普通的卷积层和后面的排列像素的步骤。就是说，最后一层卷积层输出的特征个数需要设置成固定值，即放大倍数r的平方，这样总的像素个数就与要得到的高分辨率图像一致，将像素进行重新排列就能得到高分辨率图。

在ESPCN网络中，图像尺寸放大过程的插值函数被隐含地包含在前面的卷积层中，可以自动学习到。由于卷积运算都是在低分辨率图像尺寸大小上进行，因此效率会较高。

训练时，可以将输入的训练数据标签，预处理成重新排列操作前的格式，比如将21×21的单通道图，预处理成9个通道，7×7的图，这样在训练时，就不需要做重新排列的操作。另外，ESPCN激活函数采用tanh替代了ReLU。损失函数为均方误差。

代码：[leftthomas/ESPCN](https://github.com/leftthomas/ESPCN)